In [1]:
# imports
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

class Bond(object):
    def __init__(self, coupon=None, frequency=None, maturity=None, notional=None):
        self.coupon = np.divide(coupon, 100) # coupon in rate, we assume input is in percent (eg. 4%)
        self.frequency = frequency # annual frequency is 1, semi annual frequency is 2, etc.
        self.maturity = maturity # maturity in years
        self.notional = notional # par value of the bond
        
    def present_value(self, YTM):
        # Returns: Present value in dollars given a YTM
        # We assume input is in percent (eg. 4%)
        YTM = np.divide(YTM, 100)
        # coupon * notional * 1/frequency for the coupon payment of a bond
        coupon_payment = np.prod([self.coupon, self.notional, np.divide(1, self.frequency)])
        # total number of payments is the frequency and maturity (eg. semi annual for three years is 6 payments)
        number_of_payments = np.multiply(self.maturity, self.frequency)
        # 1+ YTM/frequency
        discount_rate = np.add(1,np.divide(YTM,self.frequency))
        PV = 0
        for payment in range(1,number_of_payments+1):
            # discount for the number of terms for YTM
            total_discount_rate = np.power(discount_rate, payment)
            coupon_payment = np.prod([self.coupon, self.notional, np.divide(1, self.frequency)])
            if payment == number_of_payments:
                # you get your notional back at the end
                total_coupon_payment = np.add(coupon_payment, self.notional)
            else: 
                total_coupon_payment = coupon_payment
            PV += np.divide(total_coupon_payment, total_discount_rate)
        return PV
    
    # Generates cashflow at period X given rate Y 
    def cashflow(self, period, rate):
        rate = np.divide(rate, 100)
        if period == self.maturity:
            coupon_payment += self.notional
        discount_rate = np.add(1,np.divide(rate,self.frequency))
        total_discount_rate = np.power(discount_rate, period)
        cashflow = np.divide(coupon_payment, total_discount_rate)
        return cashflow

    def duration(self, YTM): # non modified duration, need to divide by (1+ytm/f) to get modified duration
        YTM = np.divide(YTM, 100)
        number_of_payments = np.multiply(self.maturity, self.frequency)
        coupon_payment = np.prod([self.coupon, self.notional, np.divide(1, self.frequency)])
        discount_rate = np.add(1,np.divide(YTM,self.frequency))
        PV = 0
        for payment in range(1,number_of_payments+1):
            total_discount_rate = np.power(discount_rate, payment)
            if payment == number_of_payments:
                # you get your notional back at the end
                total_coupon_payment = np.multiply(np.divide(payment, self.frequency), np.add(coupon_payment, self.notional))
            else: 
                total_coupon_payment = np.multiply(np.divide(payment, self.frequency), coupon_payment)
            PV += np.divide(total_coupon_payment, total_discount_rate)
        return np.divide(PV, self.present_value(100*YTM))
    
    def convexity(self, YTM):
        YTM = np.divide(YTM, 100)
        number_of_payments = np.multiply(self.maturity, self.frequency)
        coupon_payment = np.prod([self.coupon, self.notional, np.divide(1, self.frequency)])
        discount_rate = np.add(1,np.divide(YTM,self.frequency))
        PV = 0
        for payment in range(1,number_of_payments+1):
            total_discount_rate = np.power(discount_rate, payment)
            if payment == number_of_payments:
                # you get your notional back at the end
                total_coupon_payment = np.multiply(np.add(np.power(payment,2),payment), np.add(coupon_payment, self.notional))
            else: 
                total_coupon_payment = np.multiply(np.add(np.power(payment,2),payment), coupon_payment)
            PV += np.divide(total_coupon_payment, total_discount_rate)
            
            denominator_PV = self.present_value(100*YTM)
            denominator_second_factor = np.power((1+YTM/self.frequency),2) * np.power(self.frequency,2)
            second_derivative_factor = np.multiply(denominator_PV,denominator_second_factor)
        return np.divide(PV, second_derivative_factor)

In [4]:
bond_a = Bond(coupon=10, frequency=2, maturity=1, notional=100)
print(bond_a.present_value(5))
print(bond_a.duration(5))
print(bond_a.convexity(5))

104.81856038072576
0.9767309875141884
1.3834260321507457


In [ ]:
http://facweb.plattsburgh.edu/razvan.pascalau/BondForm.php
